In [138]:
import pandas as pd
import numpy as np
import re

In [139]:
# Load the CSV file
expedia_file_path = r"C:\Users\User\Desktop\Expedia\last_output_df_25_06_2024.csv"
booking_file_path = r"C:\Users\User\Desktop\booking\booking_hotels_clean_data_25_06.csv"
OUTPUT_FILE_PATH = r"C:\Users\User\Desktop\expedia_clean_dfs\expedia_clean_df_25_06.csv"
df = pd.read_csv(expedia_file_path)

In [140]:
print(f"expedia df shape: {df.shape}")
print(f"expedia df columns: {df.columns}")

expedia df shape: (21856, 21)
expedia df columns: Index(['Snapshot', 'start_date', 'end_date', 'name', 'TTT', 'LOS', 'score',
       'reviews', 'price_per_night', 'original_price', 'discount_perc',
       'Free_cancellation', 'No_payment', 'Breakfast', 'Option Member',
       'Index', 'star_rating', 'location_rating', 'neighborhood',
       'km_from_center', 'neighborhood_category'],
      dtype='object')


In [141]:
booking_df = pd.read_csv(booking_file_path)
print(f"booking df shape: {booking_df.shape}")
print(f"booking df columns: {booking_df.columns}")

booking df shape: (25829, 23)
booking df columns: Index(['name', 'score', 'reviews', 'room_type', 'bed_type', 'price',
       'availability', 'los', 'url', 'star_rating', 'location_rating',
       'Snapshot', 'Free_cancellation', 'No_prepayment', 'Limited_rooms',
       'TTT', 'Breakfast', 'index', 'neighborhood', 'km_from_center',
       'price_per_night', 'room_rate', 'neighborhood_category'],
      dtype='object')


In [142]:
df = df.drop(["star_rating", "location_rating", "neighborhood", "km_from_center", "neighborhood_category"], axis=1)
df.columns

Index(['Snapshot', 'start_date', 'end_date', 'name', 'TTT', 'LOS', 'score',
       'reviews', 'price_per_night', 'original_price', 'discount_perc',
       'Free_cancellation', 'No_payment', 'Breakfast', 'Option Member',
       'Index'],
      dtype='object')

In [143]:
# drop duplicates from both dataframes (booking and expedia)
subset_cols = ["name", "TTT", "LOS"]
df = df.drop_duplicates(subset=subset_cols, keep='first')
booking_df = booking_df.drop_duplicates(subset=["name", "TTT", "los"], keep='first')

In [144]:
print(f"expedia df shape: {df.shape}")
print(f"expedia df columns: {df.columns}")
print()
print(f"booking df shape: {booking_df.shape}")
print(f"expedia df columns: {booking_df.columns}")

expedia df shape: (20879, 16)
expedia df columns: Index(['Snapshot', 'start_date', 'end_date', 'name', 'TTT', 'LOS', 'score',
       'reviews', 'price_per_night', 'original_price', 'discount_perc',
       'Free_cancellation', 'No_payment', 'Breakfast', 'Option Member',
       'Index'],
      dtype='object')

booking df shape: (25826, 23)
expedia df columns: Index(['name', 'score', 'reviews', 'room_type', 'bed_type', 'price',
       'availability', 'los', 'url', 'star_rating', 'location_rating',
       'Snapshot', 'Free_cancellation', 'No_prepayment', 'Limited_rooms',
       'TTT', 'Breakfast', 'index', 'neighborhood', 'km_from_center',
       'price_per_night', 'room_rate', 'neighborhood_category'],
      dtype='object')


In [145]:
print(f"unique hotels in expedia df = {len(df['name'].unique())}")
print(f"unique hotels in booking df = {len(booking_df['name'].unique())}")

unique hotels in expedia df = 468
unique hotels in booking df = 386


In [146]:
print("Amount of combined hotels:")
print(len(df[df["name"].isin(booking_df['name'].unique())]["name"].unique()))

Amount of combined hotels:
186


In [147]:
df = df[df["name"].isin(booking_df['name'].unique())]
print(df.shape)

(10311, 16)


In [148]:
# df = df.drop(["Location grade", "Distance from center"], axis=1)

In [149]:
# change columns names
# df = df.rename(columns={'Hotel Name': "name", "Grade": "score", "Num of Reviews": "reviews", "Curr Price": "price_per_night", "Original Price": "original_price", "Percentage of discount": "discount_perc"})
# df = df.rename(columns={"Is refundable": "Free_cancellation", "Late payment": "No_payment", "Breakfast included": "Breakfast", "rating-star": "star_rating"})

In [150]:
def copy_data_from_booking_hotels(booking_df, expedia_df):
    unique_booking_hotels_data = booking_df.drop_duplicates(subset="name")
    expedia_df = expedia_df.merge(unique_booking_hotels_data[["name", "star_rating", "location_rating", "neighborhood", "km_from_center", "neighborhood_category"]], on="name", how='left')
    return expedia_df

df = copy_data_from_booking_hotels(booking_df, df)
df.columns

Index(['Snapshot', 'start_date', 'end_date', 'name', 'TTT', 'LOS', 'score',
       'reviews', 'price_per_night', 'original_price', 'discount_perc',
       'Free_cancellation', 'No_payment', 'Breakfast', 'Option Member',
       'Index', 'star_rating', 'location_rating', 'neighborhood',
       'km_from_center', 'neighborhood_category'],
      dtype='object')

In [151]:
def clean_data(df):
    # Remove non-numeric characters and convert to float for 'score'
    df['score'] = df['score'].astype(str).str.extract(r'(\d+\.\d+)')[0].astype(float)
    
    # Remove non-numeric characters for 'reviews' and handle empty strings
    df['reviews'] = df['reviews'].astype(str).str.replace(r'\D', '', regex=True)
    df['reviews'] = pd.to_numeric(df['reviews'], errors='coerce').fillna(0).astype(int)
    
    # Remove non-numeric characters for 'price_per_night' and handle empty strings
    df['price_per_night'] = df['price_per_night'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['price_per_night'] = pd.to_numeric(df['price_per_night'], errors='coerce').fillna(0.0)

    # df['price_per_night'] = df['price_per_night'] / 100
    
    # Remove non-numeric characters for 'original_price' and handle empty strings
    df['original_price'] = df['original_price'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['original_price'] = pd.to_numeric(df['original_price'], errors='coerce').fillna(0.0)
    
    # Convert 'Breakfast included' to string and update column to True/False
    df['Breakfast'] = df['Breakfast'].astype(str).str.contains('breakfast', case=False, na=False)
    
    return df

In [152]:
df = clean_data(df)

In [153]:
# Display the cleaned DataFrame
df

,Snapshot,start_date,end_date,name,TTT,LOS,score,reviews,price_per_night,original_price,...,Free_cancellation,No_payment,Breakfast,Option Member,Index,star_rating,location_rating,neighborhood,km_from_center,neighborhood_category
0,55:17.0,26/06/2024,27/06/2024,Radio Hotel,1,1,8.68,1727,194,215,...,False,False,False,False,4,4.0,NaN,Washington Heights,9.8,Uptown Manhattan
1,55:17.0,26/06/2024,27/06/2024,The Washington by LuxUrban,1,1,8.28,1018,243,270,...,False,False,False,False,6,4.0,9.4,Manhattan,7.1,Other Manhattan
2,55:17.1,26/06/2024,27/06/2024,YOTEL New York Times Square,1,1,NaN,2334,260,289,...,False,False,False,False,10,4.0,NaN,Hell's Kitchen,1.5,Midtown Manhattan
3,55:17.1,26/06/2024,27/06/2024,City Club Hotel,1,1,NaN,1353,280,329,...,True,False,False,False,14,4.0,NaN,Manhattan,1.4,Other Manhattan
4,55:17.1,26/06/2024,27/06/2024,Hyatt Place New York City/Times Square,1,1,8.08,1511,351,390,...,False,False,False,False,16,4.0,NaN,Hell's Kitchen,1.7,Midtown Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10306,42:19.2,25/07/2024,30/07/2024,The Maritime Hotel,30,5,9.49,1015,311,359,...,True,False,False,False,242,4.0,9.6,Chelsea,3.4,Midtown Manhattan
10307,42:19.3,25/07/2024,30/07/2024,Doxie Hotel,30,5,NaN,760,197,0,...,False,False,False,False,244,2.0,NaN,Manhattan,1.7,Other Manhattan
10308,42:19.4,25/07/2024,30/07/2024,The Herald by LuxUrban,30,5,8.08,224,255,284,...,True,False,False,True,247,3.0,9.6,Manhattan,2.0,Other Manhattan
10309,42:19.4,25/07/2024,30/07/2024,AC Hotel by Marriott New York Times Square,30,5,9.09,1001,268,0,...,False,False,False,False,248,4.0,NaN,Manhattan,1.6,Other Manhattan


In [154]:
# Columns to fill NaN values with the mean
columns_to_impute = ['score', 'star_rating', 'km_from_center', "reviews", "location_rating"]

# Function to fill NaN values with the mean of the column
def fill_na_with_mean(df, columns):
    for column in columns:
        mean_value = df[column].mean()
        df[column].fillna(mean_value, inplace=True)
    return df

# Fill NaN values in the specified columns
df = fill_na_with_mean(df, columns_to_impute)

# Check if NaN values have been filled
nan_summary_after = df[columns_to_impute].isna().sum()
print("NaN values in specified columns after imputation:")
print(nan_summary_after)

NaN values in specified columns after imputation:
score              0
star_rating        0
km_from_center     0
reviews            0
location_rating    0
dtype: int64


In [155]:
df.loc[df["original_price"] == 0, "discount_perc"] = 0

In [156]:
nan_summary_after = df[df.columns].isna().sum()
print("NaN values in specified columns after imputation:")
print(nan_summary_after)

NaN values in specified columns after imputation:
Snapshot                 0
start_date               0
end_date                 0
name                     0
TTT                      0
LOS                      0
score                    0
reviews                  0
price_per_night          0
original_price           0
discount_perc            0
Free_cancellation        0
No_payment               0
Breakfast                0
Option Member            0
Index                    0
star_rating              0
location_rating          0
neighborhood             0
km_from_center           0
neighborhood_category    0
dtype: int64


In [157]:
df.to_csv(OUTPUT_FILE_PATH, index=False)

In [158]:
df["neighborhood_category"].value_counts()

neighborhood_category
Other Manhattan       4185
Midtown Manhattan     3500
Downtown Manhattan    2239
Uptown Manhattan       387
Name: count, dtype: int64